<a href="https://colab.research.google.com/github/SilverStream-Cho/bigdata_class/blob/main/(2%EC%B0%A8%EC%8B%9C)_python_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1695796773000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-08-27&endtime=2023-09-26',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 9764}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.7,
  'place': '65 km W of Skwentna, Alaska',
  'time': 1695686274157,
  'updated': 1695686405660,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023cbm2b93',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023cbm2b93&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 44,
  'net': 'ak',
  'code': '023cbm2b93',
  'ids': ',ak023cbm2b93,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.83,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.7 - 65 km W of Skwentna, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-152.6321, 61.9081, 131.9]},
 'id': 'ak023cbm2b93'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.70,"65 km W of Skwentna, Alaska",1695686274157,1695686405660,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023cbm2b93,",",ak,",",origin,phase-data,",NaN,NaN,0.83,NaN,ml,earthquake,"M 1.7 - 65 km W of Skwentna, Alaska"
1,1.98,"5 km SE of Maria Antonia, Puerto Rico",1695685771710,1695688226240,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",pr71425958,",",pr,",",origin,phase-data,",7.0,0.03685,0.08,209.0,md,earthquake,"M 2.0 - 5 km SE of Maria Antonia, Puerto Rico"
2,2.21,"8 km SE of Maria Antonia, Puerto Rico",1695685713310,1695687314110,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",pr71425953,",",pr,",",origin,phase-data,",7.0,0.06625,0.09,224.0,md,earthquake,"M 2.2 - 8 km SE of Maria Antonia, Puerto Rico"
3,0.99,"2 km NNW of Anderson Springs, CA",1695685599400,1695685696506,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73941931,",",nc,",",nearby-cities,origin,phase-data,",6.0,0.02351,0.00,144.0,md,earthquake,"M 1.0 - 2 km NNW of Anderson Springs, CA"
4,4.50,"10 km S of Ishinomaki, Japan",1695685162033,1695689211040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000ky2u,",",us,",",moment-tensor,origin,phase-data,",226.0,2.46700,0.74,132.0,mwr,earthquake,"M 4.5 - 10 km S of Ishinomaki, Japan"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>